# Final Project: Intermediate SQL Querying

In [2]:
# importing the necessary libraries
import csv, sqlite3, pandas as pd

In [3]:
# Establishing database connection and cursor objects
con = sqlite3.connect("FinalDB.db")
cur = con.cursor()

In [5]:
# loading sql magic and connecting to the FinalDB database

%load_ext sql

%sql sqlite:///FinalDB.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [6]:
#downloading the datasets
from urllib.request import urlretrieve

urls = ["https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01", 
        "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01", 
        "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01"]
file_names = ["ChicagoCensusData.csv", "ChicagoPublicSchoolsData.csv", "ChicagoCrimeData.csv"]
table_names = ["Chicago_Census_Data", "Chicago_Public_Schools_Data", "Chicago_Crime_Data"]

# Storing the datasets in pandas dataframes and converting them to tables in the FinalDB database
for i, url in enumerate(urls):
    file = urlretrieve(url, file_names[i])
    df = pd.read_csv(file_names[i])
    df.to_sql(table_names[i], con, if_exists = "replace", index = False, method = "multi")

In [8]:
# retriving the created tables from the database as a confirmation

%sql SELECT name FROM sqlite_master where type='table';

 * sqlite:///FinalDB.db
Done.


name
Chicago_Census_Data
Chicago_Public_Schools_Data
Chicago_Crime_Data


In [32]:
#retrieving a list of the columns from the crimes table for reference

%sql PRAGMA table_info([Chicago_Crime_Data]);

 * sqlite:///FinalDB.db
Done.


cid,name,type,notnull,dflt_value,pk
0,ID,INTEGER,0,None,0
1,CASE_NUMBER,TEXT,0,None,0
2,DATE,TEXT,0,None,0
3,BLOCK,TEXT,0,None,0
4,IUCR,TEXT,0,None,0
5,PRIMARY_TYPE,TEXT,0,None,0
6,DESCRIPTION,TEXT,0,None,0
7,LOCATION_DESCRIPTION,TEXT,0,None,0
8,ARREST,INTEGER,0,None,0
9,DOMESTIC,INTEGER,0,None,0


## Problem 1
Finding the total number of crimes recorded in the CRIME table.

In [19]:
%sql select count(*) as Total_No_of_Crimes from Chicago_Crime_Data;

 * sqlite:///FinalDB.db
Done.


Total_No_of_Crimes
533


## Problem 2
Listing community area names and numbers with per capita income less than 11000.

In [157]:
%%sql
select community_area_number, community_area_name 
from Chicago_Census_Data where per_capita_income < 11000;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
26.0,West Garfield Park
30.0,South Lawndale
37.0,Fuller Park
54.0,Riverdale


## Problem 3
Listing all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)

In [169]:
%%sql
select case_number, description from Chicago_Crime_Data 
where description like '%minor%';

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER,DESCRIPTION
HL266884,SELL/GIVE/DEL LIQUOR TO MINOR
HK238408,ILLEGAL CONSUMPTION BY MINOR


## Problem 4
Listing all kidnapping crimes involving a child

In [74]:
%%sql 
select * from Chicago_Crime_Data where 
primary_type = 'KIDNAPPING' and description like 'child%';

 * sqlite:///FinalDB.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


## Problem 5
Listing the kind of crimes that were recorded at schools. (No repetitions)

In [161]:
%%sql 
select distinct primary_type 
from Chicago_Crime_Data where location_description 
like 'school%';

 * sqlite:///FinalDB.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


## Problem 6
Listing the type of schools along with the average safety score for each type.

In [103]:
%%sql 
select "Elementary, Middle, or High School", 
avg(safety_score) as Avg_Safety_Score 
from Chicago_Public_Schools_Data 
group by "Elementary, Middle, or High School";

 * sqlite:///FinalDB.db
Done.


"Elementary, Middle, or High School",Avg_Safety_Score
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


## Problem 7
Listing 5 community areas with highest % of households below poverty line

In [163]:
%%sql 
select community_area_name 
from Chicago_Census_Data 
order by percent_households_below_poverty 
desc nulls last limit 5;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Riverdale
Fuller Park
Englewood
North Lawndale
East Garfield Park


## Problem 8
Finding the community area that is most crime prone? Display the community area number only.

In [146]:
%%sql 
select community_area_number 
from Chicago_Crime_Data 
group by community_area_number 
order by count(*) desc nulls last limit 1;


 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER
25.0


## Problem 9
Using a sub-query to find the name of the community area with the highest hardship index

In [166]:
%%sql
select community_area_name 
from Chicago_Census_Data where hardship_index in (select max(hardship_index) from Chicago_Census_Data);

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Riverdale


## Problem 10
Using a sub-query to determine the Community Area Name with most number of crimes?

In [155]:
%%sql

select community_area_name from 
Chicago_Census_Data where community_area_number 
in (select community_area_number from 
    Chicago_Crime_Data group by 
    community_area_number order by 
    count(*) desc nulls last limit 1);

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Austin
